# SPDR/Index ETF PreProcessing

### Disclaimer: This notebook should not be considered any kind of financial advice. It exists only for the purposes of practicing modeling and making predictions

In [2]:
from statsmodels.regression.rolling import RollingOLS
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import datetime as dt
import warnings
import pandas_ta
import statsmodels.api as sm
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 25)
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [3]:
# Creating different df's for different original data

# df = pd.read_csv('/Users/samalainabayeva/Desktop/Capstone Project/SPDRs.csv')

# df = pd.read_csv('/Users/samalainabayeva/Desktop/Capstone Project/Major_Indeces.csv')

df = pd.read_csv('/Users/samalainabayeva/Desktop/Capstone Project/Indices_back_to_1964.csv')

In [4]:
df.head()

,Date,Index,Adj Close,Close,High,Low,Open,Volume
0,1964-01-02,^GSPC,75.430000,75.430000,75.790001,74.820000,0.0,4680000.0
1,1964-01-03,^GSPC,75.500000,75.500000,76.040001,75.089996,0.0,5550000.0
2,1964-01-06,^GSPC,75.669998,75.669998,76.120003,75.180000,0.0,5480000.0
3,1964-01-07,^GSPC,75.690002,75.690002,76.239998,75.250000,0.0,5700000.0
4,1964-01-08,^GSPC,76.000000,76.000000,76.349998,75.389999,0.0,5380000.0


In [50]:
# Not neccessary for BOTH of the imported csvs, only one of them

df.rename(columns={"Unnamed: 1": "Ticker"}, inplace=True)

In [5]:
# df.set_index(["Date", 'Ticker'], inplace=True)
df.set_index(["Date", 'Index'], inplace=True)

In [6]:
df.head()

,,Adj Close,Close,High,Low,Open,Volume
Date,Index,,,,,,
1964-01-02,^GSPC,75.430000,75.430000,75.790001,74.820000,0.0,4680000.0
1964-01-03,^GSPC,75.500000,75.500000,76.040001,75.089996,0.0,5550000.0
1964-01-06,^GSPC,75.669998,75.669998,76.120003,75.180000,0.0,5480000.0
1964-01-07,^GSPC,75.690002,75.690002,76.239998,75.250000,0.0,5700000.0
1964-01-08,^GSPC,76.000000,76.000000,76.349998,75.389999,0.0,5380000.0


## The following will be the same technical indicator additions as the Quant Notebook

In [7]:
# Volatility Measure

df["garman_klass_vol"] = ((np.log(df['High']) - np.log(df["Low"])) ** 2)/2 - (2*np.log(2)- 1) * (np.log(df["Adj Close"])- np.log(df['Open']))**2

In [8]:
df.head()

,,Adj Close,Close,High,Low,Open,Volume,garman_klass_vol
Date,Index,,,,,,,
1964-01-02,^GSPC,75.430000,75.430000,75.790001,74.820000,0.0,4680000.0,-inf
1964-01-03,^GSPC,75.500000,75.500000,76.040001,75.089996,0.0,5550000.0,-inf
1964-01-06,^GSPC,75.669998,75.669998,76.120003,75.180000,0.0,5480000.0,-inf
1964-01-07,^GSPC,75.690002,75.690002,76.239998,75.250000,0.0,5700000.0,-inf
1964-01-08,^GSPC,76.000000,76.000000,76.349998,75.389999,0.0,5380000.0,-inf


In [9]:
# Adding a feature for relative strength indicator -> takes 20 sessions to update

df["RSI"] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

In [10]:
df.head(1)

,,Adj Close,Close,High,Low,Open,Volume,garman_klass_vol,RSI
Date,Index,,,,,,,,
1964-01-02,^GSPC,75.43,75.43,75.790001,74.82,0.0,4680000.0,-inf,NaN


In [11]:
# These also require the length of 20 sessions to show up

# 2 STD DEVs below 20-day SMA
df['lowest_bollinger_band'] = df.groupby(level=1)["Adj Close"].transform(lambda x: pandas_ta.bbands(\
                            close=x, length=20).iloc[:,0] if pandas_ta.bbands(close=x, length=20) \
                            is not None else np.nan)

# 1 STD DEV below 20-day SMA
df['lower_bollinger_band'] = df.groupby(level=1)["Adj Close"].transform(lambda x: pandas_ta.bbands(\
                            close=x, length=20, std=1).iloc[:,0] if pandas_ta.bbands(close=x, length=20) \
                            is not None else np.nan)

# 20 - Day Simple moving average

df['20_day_SMA'] = df.groupby(level=1)["Adj Close"].transform(lambda x: pandas_ta.bbands(\
                            close=x, length=20).iloc[:,1] if pandas_ta.bbands(close=x, length=20) \
                            is not None else np.nan)

# 1 STD DEV above SMA

df['one_up_bollinger_band'] = df.groupby(level=1)["Adj Close"].transform(lambda x: pandas_ta.bbands(\
                        close=x, length=20, std=1).iloc[:,2] if pandas_ta.bbands(close=x, length=20) \
                        is not None else np.nan)

# 2 STD DEVs above SMA

df['upper_bollinger_band'] = df.groupby(level=1)["Adj Close"].transform(lambda x: pandas_ta.bbands(\
                            close=x, length=20).iloc[:,2] if pandas_ta.bbands(close=x, length=20) \
                            is not None else np.nan)

In [12]:
df.head(1)

,,Adj Close,Close,High,Low,Open,Volume,garman_klass_vol,RSI,lowest_bollinger_band,lower_bollinger_band,20_day_SMA,one_up_bollinger_band,upper_bollinger_band
Date,Index,,,,,,,,,,,,,
1964-01-02,^GSPC,75.43,75.43,75.790001,74.82,0.0,4680000.0,-inf,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Function for calculating the atr, because multiple columns needed for tranformation, .transform() will not work
# because .transform can only take 1 column as an input, we  will use a groupby().apply() function

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data["High"],
                 low=stock_data["Low"],
                 close=stock_data["Close"],
                       length=14)
    return atr

In [14]:
# Need to add group_keys = False, otherwise, it will double the date index, giving us a triple index

df["ATR"] = df.groupby(level=1, group_keys=False).apply(compute_atr)

In [15]:
df.head(1)

,,Adj Close,Close,High,Low,Open,Volume,garman_klass_vol,RSI,lowest_bollinger_band,lower_bollinger_band,20_day_SMA,one_up_bollinger_band,upper_bollinger_band,ATR
Date,Index,,,,,,,,,,,,,,
1964-01-02,^GSPC,75.43,75.43,75.790001,74.82,0.0,4680000.0,-inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Custom function for adding a new feature, utilizing pandas_ta

def compute_MACD(close):
    macd_df = pandas_ta.macd(close=close, length=20)
    
    if macd_df is not None and not macd_df.empty:
        macd = macd_df.iloc[:, 0]
        return macd
    else:
        # This else clause was very tricky to figure out, required a series of nans in the proper length,
        # which was on occassion less than 20, therefore causing errors
        return pd.Series([np.nan] * len(close), index=close.index)

In [17]:
# Moving Average Convergence Divergence

df["MACD"] = df.groupby(level=1, group_keys=False)["Adj Close"].apply(compute_MACD)

In [18]:
df.head(1)

,,Adj Close,Close,High,Low,Open,Volume,garman_klass_vol,RSI,lowest_bollinger_band,lower_bollinger_band,20_day_SMA,one_up_bollinger_band,upper_bollinger_band,ATR,MACD
Date,Index,,,,,,,,,,,,,,,
1964-01-02,^GSPC,75.43,75.43,75.790001,74.82,0.0,4680000.0,-inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Dollar volume addition

df["dollar_volume(M)"] = ((df['Adj Close']*df["Volume"])/1000000).round(4)

In [20]:
df.head()

,,Adj Close,Close,High,Low,Open,Volume,garman_klass_vol,RSI,lowest_bollinger_band,lower_bollinger_band,20_day_SMA,one_up_bollinger_band,upper_bollinger_band,ATR,MACD,dollar_volume(M)
Date,Index,,,,,,,,,,,,,,,,
1964-01-02,^GSPC,75.430000,75.430000,75.790001,74.820000,0.0,4680000.0,-inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,353.0124
1964-01-03,^GSPC,75.500000,75.500000,76.040001,75.089996,0.0,5550000.0,-inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,419.0250
1964-01-06,^GSPC,75.669998,75.669998,76.120003,75.180000,0.0,5480000.0,-inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414.6716
1964-01-07,^GSPC,75.690002,75.690002,76.239998,75.250000,0.0,5700000.0,-inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,431.4330
1964-01-08,^GSPC,76.000000,76.000000,76.349998,75.389999,0.0,5380000.0,-inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,408.8800


In [21]:
# SPDRS

df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 45496 entries, ('1964-01-02', '^GSPC') to ('2023-11-02', '^RUT')
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Adj Close              45496 non-null  float64
 1   Close                  45496 non-null  float64
 2   High                   45496 non-null  float64
 3   Low                    45496 non-null  float64
 4   Open                   45496 non-null  float64
 5   Volume                 45496 non-null  float64
 6   garman_klass_vol       45496 non-null  float64
 7   RSI                    45416 non-null  float64
 8   lowest_bollinger_band  45420 non-null  float64
 9   lower_bollinger_band   45420 non-null  float64
 10  20_day_SMA             45420 non-null  float64
 11  one_up_bollinger_band  45420 non-null  float64
 12  upper_bollinger_band   45420 non-null  float64
 13  ATR                    45440 non-null  float64
 14  MACD           

# DONT RUN NEXT CELL!!!

In [45]:
# INDEX LEVEL

df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 31068 entries, ('1992-12-22', '^DJI') to ('2023-10-25', '^RUT')
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Adj Close              31068 non-null  float64
 1   Close                  31068 non-null  float64
 2   High                   31068 non-null  float64
 3   Low                    31068 non-null  float64
 4   Open                   31068 non-null  float64
 5   Volume                 31068 non-null  int64  
 6   garman_klass_vol       31068 non-null  float64
 7   RSI                    30988 non-null  float64
 8   lowest_bollinger_band  30992 non-null  float64
 9   lower_bollinger_band   30992 non-null  float64
 10  20_day_SMA             30992 non-null  float64
 11  one_up_bollinger_band  30992 non-null  float64
 12  upper_bollinger_band   30992 non-null  float64
 13  ATR                    31012 non-null  float64
 14  MACD            

In [22]:
# Cell for creating different files for use

# df.to_csv("/Users/samalainabayeva/Desktop/Capstone Project/SPDRs_w_technical_indicators.csv")
# df.to_csv("/Users/samalainabayeva/Desktop/Capstone Project/INDECES_w_technical_indicators.csv")
df.to_csv('/Users/samalainabayeva/Desktop/Capstone Project/Indices_back_to_1964_with_indicators.csv')